In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [36]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler,OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,plot_confusion_matrix,recall_score,precision_score
import seaborn as sns
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

In [37]:
import pandas as pd

In [38]:
data = pd.read_csv('/kaggle/input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv')
data.head()

In [39]:
print(data)

In [40]:
data.info

In [41]:
data.describe()

In [42]:
data.isnull().sum()

In [43]:
data.shape

In [44]:
data.status.value_counts()

In [45]:
data.salary = data.salary.fillna(0)
data.isnull().sum()

In [46]:
sns.heatmap(data.corr(),annot=True)

In [47]:
sns.distplot(data.salary)

In [48]:
sns.countplot(data.gender)

In [49]:
sns.countplot(data.hsc_s)

In [50]:
only_sci = data[data.hsc_s=="Science"]
sns.countplot(only_sci.degree_t)

In [51]:
sns.countplot(only_sci[only_sci.degree_t=="Others"].specialisation)


In [52]:
sns.pairplot(data,hue='specialisation')

In [53]:
sns.pairplot(data=data, hue='gender')

In [54]:
data.groupby('status').sum()

In [55]:
data.groupby('specialisation').mean()['salary'].sort_values(ascending=False)

In [56]:
data.groupby('hsc_s').count()['status'].sort_values(ascending=False)

In [57]:
data.groupby('hsc_s').mean()['salary'].sort_values(ascending=False)

In [58]:
sns.regplot(data.degree_p,data.salary)

In [59]:
sns.regplot(data.ssc_p,data.salary)

In [60]:
sns.regplot(data.hsc_p,data.salary)

In [61]:
data.groupby('ssc_b').count()['status'].sort_values(ascending=False)

In [62]:
data.groupby('ssc_b').mean()['salary'].sort_values(ascending=False)

In [63]:
data.groupby('workex').count()['status'].sort_values(ascending=False)

In [64]:
data.groupby('workex').mean()['salary'].sort_values(ascending=False)

In [65]:
sns.boxplot(data=data)

In [66]:
data = data.drop(columns=['sl_no','salary'])
x = data.iloc[:,:-1]
x.head()

In [67]:
y=data[['status']]
y.head()

In [68]:
y.status = LabelEncoder().fit_transform(y.status)
y.head()

In [69]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)

In [70]:
x_test.shape

In [71]:
x_train.head()

In [72]:
trans_nom = make_column_transformer((OneHotEncoder(sparse=False),[0,5,7,10]),
                                    (OrdinalEncoder(),[2,4,8]),
                                    remainder="passthrough")

In [73]:
trans_nom.fit_transform(x_train)

In [74]:
model = LogisticRegression(solver='liblinear')

In [75]:
set_config(display='diagram')
pipe = make_pipeline(trans_nom, model)
pipe

In [76]:
data.gender.unique()

In [77]:
pipe.fit(x_train,y_train)

In [78]:
x_train.shape

In [79]:
pred = pipe.predict(x_test)
accuracy_score(pred,y_test)*100